In [35]:
import pandas as pd
import sqlite3 as sql

In [36]:
#Import the data base
CHICAGO_CRIME_DATA = pd.read_csv("/Users/alneagu/Git/SQL_Projects/ChicagoCrimeData.csv")
CENSUS_DATA = pd.read_csv("/Users/alneagu/Git/SQL_Projects/ChicagoCensusData.csv")
CHICAGO_PUBLIC_SCHOOLS = pd.read_csv("/Users/alneagu/Git/SQL_Projects/ChicagoPublicSchools.csv")

#Set the dataframe width 
pd.set_option("display.max_colwidth",None)

In [37]:
data_base = sql.connect("/Users/alneagu/Git/SQL_Projects/RealWorldData.db")
c = data_base.cursor()

In [38]:
CHICAGO_CRIME_DATA.to_sql("CHICAGO_CRIME_DATA", data_base, if_exists='replace', index=False,method="multi")
CENSUS_DATA.to_sql("CENSUS_DATA", data_base, if_exists='replace', index=False,method="multi")
CHICAGO_PUBLIC_SCHOOLS.to_sql("CHICAGO_PUBLIC_SCHOOLS", data_base, if_exists='replace', index=False,method="multi")


566

1. Find the number of crimes recorded in CRIME table

In [39]:
#1. Find the number of crimes recorded in CRIME table
query1 = '''
SELECT COUNT(*) 
FROM CHICAGO_CRIME_DATA
'''
pd.read_sql_query(query1, data_base)



,COUNT(*)
0,533


2. List community areas with per capita income less than 11000.

In [41]:
query2 = '''
SELECT COMMUNITY_AREA_NAME, per_capita_income
FROM CENSUS_DATA
WHERE per_capita_income < 11000;
'''
pd.read_sql_query(query2, data_base)

,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
0,West Garfield Park,10934
1,South Lawndale,10402
2,Fuller Park,10432
3,Riverdale,8201


3. List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [43]:
query3 = '''
SELECT id, case_number, description
FROM CHICAGO_CRIME_DATA
WHERE description LIKE "%MINOR";
'''
pd.read_sql_query(query3, data_base)

,CASE_NUMBER,DESCRIPTION
0,HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
1,HK238408,ILLEGAL CONSUMPTION BY MINOR


4. List all kidnapping crimes involving a child

In [44]:
query4 = '''
SELECT id, case_number, primary_type, description
FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE ="KIDNAPPING" AND DESCRIPTION LIKE "%CHILD%";
'''
pd.read_sql_query(query4, data_base)

,ID,CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
0,5276766,HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


5. What kind of crimes were recorded at schools?

In [47]:
query5 = '''
SELECT DISTINCT(primary_type) as CRIME_TYPE
FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE "%SCHOOL%";
'''
pd.read_sql_query(query5, data_base)

,CRIME_TYPE
0,BATTERY
1,CRIMINAL DAMAGE
2,NARCOTICS
3,ASSAULT
4,CRIMINAL TRESPASS
5,PUBLIC PEACE VIOLATION


6. List the average safety score for all types of schools

In [48]:
query6 = '''
SELECT AVG(SAFETY_SCORE), "Elementary, Middle, or High School"
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School";
'''
pd.read_sql_query(query6, data_base)

,AVG(SAFETY_SCORE),"Elementary, Middle, or High School"
0,49.520384,ES
1,49.623529,HS
2,48.000000,MS


7. List 5 community areas with the highest % of households below poverty line

In [56]:
query7 = '''
SELECT COMMUNITY_AREA_NAME, community_area_number,PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC  LIMIT 5;
'''
pd.read_sql_query(query7, data_base)

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,54.0,56.5
1,Fuller Park,37.0,51.2
2,Englewood,68.0,46.6
3,North Lawndale,29.0,43.1
4,East Garfield Park,27.0,42.4


8. Which community area is most crime prone?

In [58]:
query8 = '''
SELECT 
    COUNT(crime.community_area_number) as NUM_OF_CRIMES, crime.COMMUNITY_AREA_NUMBER, census.COMMUNITY_AREA_NAME
FROM CHICAGO_CRIME_DATA AS crime
JOIN 
    CENSUS_DATA AS census 
ON
    crime.COMMUNITY_AREA_NUMBER = census.COMMUNITY_AREA_NUMBER
GROUP BY crime.community_area_number
ORDER BY NUM_OF_CRIMES DESC LIMIT 1
'''
pd.read_sql_query(query8, data_base)

,NUM_OF_CRIMES,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,43,25.0,Austin


9. Use a sub-query to find the name of the community area with highest hardship index.

In [64]:
query9 = '''
SELECT community_area_name, hardship_index
FROM CENSUS_DATA
WHERE hardship_index = (SELECT MAX(hardship_index) FROM CENSUS_DATA)
'''
pd.read_sql_query(query9, data_base)

,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
0,Riverdale,98.0


10. Use a sub-query to determine the Community Area Name with most number of crimes?

In [71]:
query10= '''
SELECT community_area_name, community_area_number
FROM CENSUS_DATA
WHERE 
    community_area_number = (
        SELECT  community_area_number 
        FROM CHICAGO_CRIME_DATA crime
        GROUP BY community_area_number
        ORDER BY COUNT(community_area_number) DESC LIMIT 1
        )
    
'''
pd.read_sql_query(query10, data_base)

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
0,Austin,25.0
